# PCA + SVM

---
---
## Libraries

In [5]:
%matplotlib widget
from ipywidgets import AppLayout, TwoByTwoLayout, IntSlider, FloatSlider
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from IPython.display import clear_output
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import ipywidgets as widgets
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import time

# import umap

In [8]:
# score=[1,2,3,4,6,7,10,15,4.6,3]
# score2=[1,2,3,4,6,7,10,15,4.6,3]
# plt.ion()
# plt.scatter(score,score2)
# plt.xlabel("Dimensions")
# plt.ylabel("Score")
# plt.show()

---
---
## Code

### Retrieving Data

In [2]:
def data_loader(file='MCF7',section='processed', timer=False):
    start = time.time()
#>> Import, Rename, Cleaning Data (Missing XCells train data)
    def renamer(name):
        classification = name.split("_")[-1] #change -1 into -3
        cell = name.split("_")[-2]
        if len(classification) > 4:
            classification = classification[:4]
        return classification+"_"+cell

    # Train
    filepath_Train = f"{section}_data(DropSeq)\\{file}_SmartS_Filtered_Standardized_Normalised_3000_Data_train.txt" # remove "(DropSeq)"
    df_Train = pd.read_csv(filepath_Train,delimiter="\ ",engine='python',index_col=0)
    df_Train.rename(mapper=renamer, axis='columns', inplace=True)
    df_Train.dropna(axis='rows', inplace = True)
    # print(df_Train.shape)

    # Test
    filepath_Test = f"{section}_data(DropSeq)\\{file}_SmartS_Filtered_Standardized_Normalised_3000_Data_test.txt" # remove "(DropSeq)"
    df_Test = pd.read_csv(filepath_Test,delimiter="\ ",engine='python',index_col=0)
    df_Test.rename(mapper=renamer, axis='columns', inplace=True)
    df_Test.dropna(axis='rows', inplace = True)
    # print(df_Test.shape)
    max_dim = min(df_Train.shape)

    end = time.time()
    if timer:
        print(f"Import time: {end-start:.6f}s")
    return df_Train, df_Test, max_dim

### Hyperparameter Tuning

In [3]:
def PCA_SVC_Hyper_tuning(file='MCF7', fold=5, section='processed', timer=False, verbose=True):
    X_train,X_test,max_dim=data_loader(file=file, section=section, timer=timer)
    max_dim=(max_dim//fold)*(fold-1)
#>> Pipeline
    pipe = Pipeline(steps=[
                ("scaler", StandardScaler()), 
                ("pca", PCA()),
                ("svc", SVC()),
            ])
    param_grid ={
                "pca__n_components": [d for d in range(max_dim, max_dim+1)], # change max_dim into 2
                # "svc__C": [c/1000 for c in range(1,10)],
                # "svc__gamma": ('scale', 'auto'),
                # "svc__kernel": ('linear', 'rbf', 'poly', 'sigmoid'),
            } 

    y = [int(i.split("_")[0]=='Norm') for i in X_train.columns]

    clf = GridSearchCV(pipe, param_grid, n_jobs=2, cv=fold)
    clf.fit(X_train.T, y)

    table = pd.DataFrame(clf.cv_results_)
    i = clf.best_index_
    best = table[i:i+1]
    cv_results = pd.concat((best.set_index('rank_test_score'),table.drop(index=i).set_index('rank_test_score').sort_index()))

    y_test = [int(i.split("_")[0]=='Norm') for i in X_test.columns]
    if verbose:
        print(f"best parameters: {clf.best_params_}")
        print(f"best score: {clf.best_score_:.3f}")
        print(f"prediction score: {clf.score(X_test.T,y_test):.3f}")
        display(cv_results.head())
    cv_results.to_csv(f"./CV-Scores/DropSeq_{file}_{fold}-fold") # remove "(DropSeq)"

    return clf

### Visualization

In [4]:
def PCA_SVC(df_Train, df_Test, d, c, kernel, timer=False, graph=False):

    # d = parameters['pca__n_components']
    # c = parameters['svc__C']
    # gamma = parameters['svc__gamma']
    # kernel = parameters['svc__kernel']

    start = time.time()
#>> PCA 
    pca = Pipeline((
                ("scaler", StandardScaler()), # only this one is good with only filtered data for linear kernel opposite for rbf
                # ('normalizer',MinMaxScaler()),  # helps with poly kernel
                ("pca", PCA(n_components=d)),
            ))
    start_PCA = time.time()
    pca.fit(df_Train.T)
    mid_PCA = time.time()
    df_Train_reduced = pd.DataFrame(pca.transform(df_Train.T), columns=pca["pca"].get_feature_names_out(), index=df_Train.columns)
    end_PCA = time.time()
    df_Train_visual = df_Train_reduced.loc[:,["pca0","pca1"]]

    # pca.fit(df_Test.T)    # do not re-fit ?
    df_Test_reduced = pd.DataFrame(pca.transform(df_Test.T), columns=pca["pca"].get_feature_names_out(), index=df_Test.columns) #<<< use transform insted of fit_tranform
    df_Test_visual = df_Test_reduced.loc[:,["pca0","pca1"]]

#>> Pipeline

    X = df_Train_reduced
    y = [int(i.split("_")[0]=='Norm') for i in df_Train.columns]

    svm_clf = Pipeline((
            # ("scaler", StandardScaler()),
            # ('normalizer',MinMaxScaler()),
            ("linear_svc", SVC(C=c, kernel=kernel)),
        ))
    start_SVC = time.time()
    clf = svm_clf.fit(X, y)
    mid_SVC = time.time()
    y_pred = clf.predict(df_Test_reduced)
    mid2_SVC = time.time()
    y_test = [int(i.split("_")[0]=='Norm') for i in df_Test.columns]
    score = clf.score(df_Test_reduced, y_test)
    end_SVC = time.time()

    fig, supp = None, None
    if graph:
#>> Graph
        plt.ioff()
        fig = plt.figure('PCA_SVC')
        plt.cla()

        ax = plt.gca()

        if d==2:
            DecisionBoundaryDisplay.from_estimator(   # Plots decision boundary
                clf,
                df_Train_visual,
                eps = 10,
                ax=ax,
                grid_resolution=50,
                plot_method="contour",
                colors="k",
                levels=[-1, 0, 1],
                alpha=0.5,
                linestyles=["--", "-", "--"],
            )

        # Plotting Train
        np_Norm = df_Train_visual.loc[[i for i in df_Train_visual.index if 'Norm'==i.split("_")[0]]].to_numpy()
        np_Hypo = df_Train_visual.loc[[i for i in df_Train_visual.index if 'Hypo'==i.split("_")[0]]].to_numpy()
        plt.scatter(np_Norm[:, 0], np_Norm[:, 1], c='g', s=10, marker="s", label='Normoxia')
        plt.scatter(np_Hypo[:, 0], np_Hypo[:, 1], c='b', s=10, marker="^", label='Hypoxia')

        # Plotting Test
        np_Norm_Predicted = df_Test_visual.iloc[[i for i in range(len(y_pred)) if y_pred[i]==1 and y_pred[i]==y_test[i]]].to_numpy()
        np_Norm_Miss = df_Test_visual.iloc[[i for i in range(len(y_pred)) if y_pred[i]==1 and y_pred[i]!=y_test[i]]].to_numpy()
        np_Hypo_Predicted = df_Test_visual.iloc[[i for i in range(len(y_pred)) if y_pred[i]==0 and y_pred[i]==y_test[i]]].to_numpy()
        np_Hypo_Miss = df_Test_visual.iloc[[i for i in range(len(y_pred)) if y_pred[i]==1 and y_pred[i]!=y_test[i]]].to_numpy()
        plt.scatter(np_Norm_Predicted[:, 0], np_Norm_Predicted[:, 1], c='g',edgecolors='k', s=40, marker="X", label='Normoxia Test')
        plt.scatter(np_Norm_Miss[:, 0], np_Norm_Miss[:, 1], c='g',edgecolors='r', s=40, marker="X")
        plt.scatter(np_Hypo_Predicted[:, 0], np_Hypo_Predicted[:, 1], c='b',edgecolors='k', s=40, marker="X", label='Hypoxia Test')
        plt.scatter(np_Hypo_Miss[:, 0], np_Hypo_Miss[:, 1], c='b',edgecolors='r', s=40, marker="X")
        if len(np_Hypo_Miss)!=0 or len(np_Norm_Miss)!=0:
            plt.scatter([],[], c='none', edgecolors='r', s=40, marker="X", label='Misclassified')

        # Highligting support Vectors
        supp = np.array(X)[svm_clf["linear_svc"].support_]

        plt.scatter(      #highlights support vector
        supp[:, 0],
        supp[:, 1],
        s=50,
        linewidth=0.7,
        facecolors="none",
        # alpha=.1,
        edgecolors="r",
        label='Support'
        )

        ax.set_xlabel('PCA 1')
        ax.set_ylabel('PCA 2')

        plt.legend(loc=1)
    end = time.time()

    if timer:
        print(f"Running time: {end-start:.6f}s")
        print(f"> PCA:\n\tTraining: {mid_PCA-start_PCA:.6f}s\n\tReduction: {end_PCA-mid_PCA:.6f}s")
        print(f"> SVC:\n\tTraining: {mid_SVC-start_SVC:.6f}s\n\tPrediction: {mid2_SVC-mid_SVC:.6f}s\n\tScoring: {end_SVC-mid2_SVC:.6f}s")
        if graph:
            print(f"\tGraph: {end-end_SVC:.6f}s")

    return {'graph': fig, 'model': clf, 'max dim': df_Train.shape[1], 'support': supp, 'score':score, 'runtime':f"{end-start:.6f}"}
# X_train,X_test,max_dim=data_loader(file='MCF7')
# out = PCA_SVC(X_train,X_test,15,graph=True)
# display(out['graph'])

In [5]:
def widget_PCA_SVC(model, file='MCF7'): 
    plt.ioff()

    c=model.best_params_['svc__C']
    d=model.best_params_['pca__n_components']
    kernel=model.best_params_['svc__kernel']
    
    file_type = widgets.Dropdown(
        options=['HCC1806', 'MCF7', 'XCells'],
        value=file,
        description='File:',
    )

    kernel_type = widgets.Dropdown(
        options=['linear', 'poly', 'rbf', 'sigmoid'],
        value=kernel,
        description='Kernel:',
    )

    dim_slider = IntSlider(
        orientation='horizontal',
        description='Dimensions:',
        value=d,
        min=2,
        step=1
    )

    c_slider = FloatSlider(
        orientation='horizontal',
        description='C Value:',
        value=c,
        min=0.001,
        max=10*c,
        step=0.001,
        readout_format='.3f'
    )

    out = widgets.Output()

    dim_slider.layout.width = '40%'
    c_slider.layout.width = '40%'

    X_train,X_test,max_dim=data_loader(file=file_type.value)
    svm_output = PCA_SVC(X_train,X_test,d=dim_slider.value, c=c_slider.value, kernel=kernel_type.value, graph=True)
    dim_slider.max = max_dim
    fig = svm_output['graph']

    with out:
        print("Score: {:.3f}".format(svm_output['score']))
        print(str(len(svm_output['support']))+" support vectors")
    fig.canvas.resizable = False
    # fig.canvas.header_visible = False
    # fig.canvas.toolbar_position = 'top'
    fig.canvas.layout.min_height = '400px'
    plt.title(file_type.value+' | {} kernel | Dim = {} | C = {:.3f}'.format(kernel_type.value, dim_slider.value, c_slider.value))
    fig.canvas.draw()
    fig.canvas.flush_events()

    def update(change):
        if file_type.value == 'XCells':
            out.clear_output()
            file_type.value = change.old
            with out:
                print("\n! - '"+change.new+"' file missing in folder 'raw_data\\' - !\n")
            return
        X_train,X_test,max_dim=data_loader(file=file_type.value)
        if dim_slider.value > max_dim:
            dim_slider.value = max_dim
        dim_slider.max = max_dim

        svm_output = PCA_SVC(X_train,X_test,d=dim_slider.value, c=c_slider.value, kernel=kernel_type.value, graph=True)
        support = svm_output['support']
        fig = svm_output['graph']
        out.clear_output()
        with out:
            print("Score: {:.3f}".format(svm_output['score']))
            print(str(len(support))+" support vectors")
        plt.title('{} | {} kernel | Dim = {} | C = {:.3f}'.format(file_type.value, kernel_type.value, dim_slider.value, c_slider.value))
        fig.canvas.draw()
        fig.canvas.flush_events()


    dim_slider.observe(update, names='value')
    c_slider.observe(update, names='value')
    file_type.observe(update, names='value')
    kernel_type.observe(update, names='value')

    dropdowns = TwoByTwoLayout(top_left=file_type,
                             bottom_left=kernel_type,
                             merge=False)
    
    sliders = TwoByTwoLayout(top_left=dim_slider,
                             bottom_left=c_slider)

    layout = AppLayout(
        left_sidebar=fig.canvas,
        center=out,
        footer=sliders,
        pane_heights=[0, 6, 1],
        pane_widths=[6, 6, 3]
    )
    display(dropdowns)
    display(layout)

---
---
## Output

### Hyperparameter Tuning

In [6]:
model_MCF = PCA_SVC_Hyper_tuning('MCF7', 2)
model_HCC = PCA_SVC_Hyper_tuning('HCC1806', 2)
# SmartS 20-fold ~460m

best parameters: {'pca__n_components': 1500}
best score: 0.962
prediction score: 0.968


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__n_components,params,split0_test_score,split1_test_score,mean_test_score,std_test_score
rank_test_score,,,,,,,,,,
1,69.946463,0.414331,57.335205,0.002116,1500,{'pca__n_components': 1500},0.962201,0.962543,0.962372,0.000171


best parameters: {'pca__n_components': 1500}
best score: 0.924
prediction score: 0.937


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__n_components,params,split0_test_score,split1_test_score,mean_test_score,std_test_score
rank_test_score,,,,,,,,,,
1,42.399369,0.070896,33.98793,0.070821,1500,{'pca__n_components': 1500},0.925421,0.923548,0.924485,0.000936


### Visualization

In [8]:
# widget_PCA_SVC(model_HCC, file='HCC1806')

In [ ]:
# # runtime ~5m
# plt.ioff()
# for file in ['HCC1806','MCF7']:
#     print(file)
#     X_train,X_test,max_dim=data_loader(file=file, timer=True)
#     max_dim=100
#     for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
#         score=[]
#         start=time.time()
#         for d in range(1,max_dim):
#             score.append(PCA_SVC(X_train,X_test, d=20, c=d/10, kernel=kernel)['score'])
#         plt.cla()
#         plt.plot([d/10 for d in range(1,max_dim)],score)
#         ax = plt.gca()
#         ax.set_xlabel('C value (d=20)')
#         ax.set_ylabel('Score')
#         avg_score = np.sum(score)/(len(score))
#         plt.title(file+'_'+kernel+f' | avg. score: {avg_score:.3f}')
#         plt.savefig("./Score-plain_ignore/{}_{}.png".format(file, kernel))
#         print(f"\tfinished {kernel} kernel in: {time.time()-start:.1f}s   \taverage score: {avg_score:.3f}")
#     print("----------------------------------------------")